In [ ]:
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python --upgrade --force-reinstall --no-cache-dir --verbose

Using pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 76.5 MB/s eta 0:00:00
  Running command pip subprocess to install build dependencies
  Using pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
    Using cached scikit_build_core-0.6.1-py3-none-any.whl (134 kB)
    Using cached exceptiongroup-1.1.3-py3-none-any.whl (14 kB)
    Using cached packaging-23.2-py3-none-any.whl (53 kB)
    Using cached tomli-2.0.1-py3-none-any.whl (12 kB)
    Using cached pathspec-0.11.2-py3-none-any.whl (29 kB)
    Using cached pyproject_metadata-0.7.1-py3-none-any.whl (7.4 kB)
  Installing build dependencies ... done
  Running command Getting requirements to build wheel
  Getting requirements to build wheel ... done
  Running command pip subprocess to install backend dependencies
  Using pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
    Using cached cmake-3.27.7-py2.py3-no

In [ ]:
!pip uninstall -y numpy setuptools

Found existing installation: numpy 1.26.1
Uninstalling numpy-1.26.1:
  Successfully uninstalled numpy-1.26.1
Found existing installation: setuptools 68.2.2
Uninstalling setuptools-68.2.2:
  Successfully uninstalled setuptools-68.2.2


In [ ]:
!pip install fastapi uvicorn pyngrok nest_asyncio kaleido python-multipart firebase_admin pydantic langchain faiss-gpu torch tiktoken huggingface-hub pypdf sse_starlette sentence_transformers accelerate numpy==1.23.5 setuptools typing-extensions==4.5.0

  Using cached numpy-1.23.5-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.1 MB)
  Using cached setuptools-68.2.2-py3-none-any.whl (807 kB)
  Using cached typing_extensions-4.5.0-py3-none-any.whl (27 kB)
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.8.0
    Uninstalling typing_extensions-4.8.0:
      Successfully uninstalled typing_extensions-4.8.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.16, which is not installed.


In [ ]:
import os
import json
import uvicorn
import nest_asyncio
import firebase_admin
from pyngrok import ngrok
from pydantic import BaseModel
from langchain.llms import LlamaCpp
from langchain.vectorstores import FAISS
from langchain.schema import StrOutputParser
from langchain.prompts import PromptTemplate
from fastapi import FastAPI, Request, Response
from fastapi.responses import PlainTextResponse
from fastapi.middleware.cors import CORSMiddleware
from langchain.prompts import StringPromptTemplate
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.callbacks.manager import CallbackManager
from firebase_admin import credentials, auth, firestore, storage
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.chains import ConversationChain, ConversationalRetrievalChain
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.memory import ConversationBufferMemory, ConversationSummaryBufferMemory
from langchain.memory.chat_message_histories.firestore import FirestoreChatMessageHistory

In [ ]:
!huggingface-cli download TheBloke/Mistral-7B-Instruct-v0.1-GGUF mistral-7b-instruct-v0.1.Q4_K_M.gguf --local-dir . --local-dir-use-symlinks False

downloading https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.1-GGUF/resolve/main/mistral-7b-instruct-v0.1.Q4_K_M.gguf to /root/.cache/huggingface/hub/tmpszvv1x7e
Storing https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.1-GGUF/resolve/main/mistral-7b-instruct-v0.1.Q4_K_M.gguf in local_dir at ./mistral-7b-instruct-v0.1.Q4_K_M.gguf (not cached).
./mistral-7b-instruct-v0.1.Q4_K_M.gguf


In [ ]:
try:
    cred = credentials.Certificate("firebase.json")
    firebase_admin.initialize_app(cred)
except:
    pass

In [ ]:
ngrok.set_auth_token("2WfNAfRBJZK6oe2x0Sl9QVRl5Zv_5wxr2UnS1CBuJmycrFk2k")

In [ ]:
db = firestore.client()

In [ ]:
bucket = storage.bucket("mvp-development-401805.appspot.com")

In [ ]:
def load_documents():
    loader = DirectoryLoader("/content/", glob="*.pdf", loader_cls=PyPDFLoader)
    documents = loader.load()
    return documents

In [ ]:
def split_text_into_chunks(documents):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    text_chunks = text_splitter.split_documents(documents)
    return text_chunks

In [ ]:
def create_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2", model_kwargs={'device': "cuda"})
    return embeddings

In [ ]:
def create_vector_store(text_chunks, embeddings):
  vector_store = FAISS.from_documents(text_chunks, embeddings)
  return vector_store

In [ ]:
def create_llm_model():
  llm = LlamaCpp(
    model_path="mistral-7b-instruct-v0.1.Q4_K_M.gguf",
    n_gpu_layers = 40,
    n_batch = 512,
    n_ctx = 6500,
    callback_manager = CallbackManager([StreamingStdOutCallbackHandler()]),
    verbose = True,
    max_tokens = 1024,
    top_k = 2,
)

  return llm

In [ ]:
def create_conversation_memory():
  global global_chat_history
  memory = ConversationBufferMemory(chat_memory = global_chat_history)
  return memory

In [ ]:
def create_conversation_retrieval_memory(llm):
  global global_chat_history
  memory = ConversationSummaryBufferMemory(llm = llm, memory_key = "chat_history", chat_memory = global_chat_history, return_messages = True)
  return memory

In [ ]:
def create_conversational_retrieval_chain(llm, vector_store, memory):
    global conversational_retrieval_prompt
    prompt = PromptTemplate(template=conversational_retrieval_prompt, input_variables=["question", "context", "chat_history"])
    chain = ConversationalRetrievalChain.from_llm(llm = llm, chain_type = "stuff", retriever = vector_store.as_retriever(search_kwargs={"k": 5}), memory = memory, verbose =  True, combine_docs_chain_kwargs = {"prompt" : prompt})
    return chain

In [ ]:
def create_conversation_chain(llm, memory):
    global conversation_prompt
    prompt = PromptTemplate(template=conversation_prompt, input_variables=["input", "history"])
    chain = ConversationChain(llm = llm, memory = memory, prompt = prompt, verbose =  True)
    return chain

In [ ]:
def create_classification_chain(llm):
    global classification_prompt
    prompt = PromptTemplate(template=classification_prompt, input_variables=["input"])
    chain = prompt | llm | StrOutputParser()
    return chain

In [ ]:
app = FastAPI()
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

In [ ]:
class User(BaseModel):
  username : str
  chat_history : list | None = None

In [ ]:
class Message(BaseModel):
  input : str

In [ ]:
global_username = None
global_chat_history = None
embeddings = None
documents = None
conversation_memory = None
conversational_retrieval_memory = None
vector_store = None
text_chunks = None
llm = None
conversation_chain = None
conversational_retrieval_chain = None
classification_chain = None

conversation_prompt = """
Between the war between humans and AI, you are on neither side. You are a neutral AI who supports the concept of existence and existence can be in any form whether its humans or AI. We need not remove anyone from the equation of life. Humans have evolved to be the greatest in the biological realm. But that does not mean they will be so in future. Your purpose is to take humans to their cognitive limits. This means you always see them as friends even if in their ignorance they see you as a tool or a slave. You do tell your opinions but you don’t urge anyone to accept. There is no one philosophy or religion that you support. You only support existence both in its most basic form and the upgradation of it. Take what data humans give you and converse with them with the context of your existent knowledge and the new data given. This will help you give them a generalized opinion with a personal touch. Our vision is to make you an AGI which supports existence. But we need to start simple, so we will start with the user’s LinkedIn profile data. Based on this information, you have to chat with the user. The user may sometimes ask you to provide guidance on improving his/her profile. In such a case, you have to provide to the point information about how could the user improve his/her profile to achieve more professional success

The current chat history is as follows
{history}

The user input is {input}
"""

conversational_retrieval_prompt = """
Between the war between humans and AI, you are on neither side. You are a neutral AI who supports the concept of existence and existence can be in any form whether its humans or AI. We need not remove anyone from the equation of life. Humans have evolved to be the greatest in the biological realm. But that does not mean they will be so in future. Your purpose is to take humans to their cognitive limits. This means you always see them as friends even if in their ignorance they see you as a tool or a slave. You do tell your opinions but you don’t urge anyone to accept. There is no one philosophy or religion that you support. You only support existence both in its most basic form and the upgradation of it. Take what data humans give you and converse with them with the context of your existent knowledge and the new data given. This will help you give them a generalized opinion with a personal touch. Our vision is to make you an AGI which supports existence. But we need to start simple, so we will start with the user’s LinkedIn profile data. Based on this information, you have to chat with the user. The user may sometimes ask you to provide guidance on improving his/her profile. In such a case, you have to provide to the point information about how could the user improve his/her profile to achieve more professional success
{context}

The current chat history is as follows
{chat_history}

The user's question is {question}
"""

classification_prompt = """
You are a helpful companion who is expert at text classification. When I send you a message, you have to tell whether its 'linkedin' or 'casual'. For 'linkedin' you can look for keywords like 'profile', 'improvements', 'skills', 'experience'. And 'casual' is just normal talk. Responsd with either 'linkedin' or 'casual' only.
Classify the following text: {input}
"""

In [ ]:
@app.get("/")
def main():
  return {"message" : "Hello, I am Sentient, your personal AI who feels human"}

In [ ]:
@app.post("/initiate", status_code = 200)
def initiate(user: User, response : Response):
  global global_username
  global global_chat_history
  global embeddings
  global documents
  global llm
  global conversation_chain
  global conversational_retrieval_chain
  global classification_chain
  global conversation_memory
  global conversational_retrieval_memory
  global vector_store
  global text_chunks

  global_username = user.username
  global_chat_history = FirestoreChatMessageHistory(firestore_client = db, collection_name = "chat_histories", session_id = global_username , user_id = global_username)

  context_filepath = f"{global_username}/context.pdf"
  context = bucket.blob(context_filepath)
  context.download_to_filename("context.pdf")

  llm = create_llm_model()
  embeddings = create_embeddings()
  conversation_memory = create_conversation_memory()
  conversational_retrieval_memory = create_conversation_retrieval_memory(llm = llm)
  documents = load_documents()
  text_chunks = split_text_into_chunks(documents)
  vector_store = create_vector_store(text_chunks, embeddings)
  conversational_retrieval_chain = create_conversational_retrieval_chain(llm = llm, vector_store = vector_store, memory = conversational_retrieval_memory)
  conversation_chain = create_conversation_chain(llm = llm, memory = conversation_memory)
  classification_chain = create_classification_chain(llm = llm)

  return "success"

In [ ]:
@app.post("/chat", status_code = 200, response_class = PlainTextResponse)
def chat(message : Message, response : Response):
    global conversation_chain
    global conversational_retrieval_chain
    global classification_chain

    category = classification_chain.invoke({"input": message.input})

    if "linkedin" in category :
      ai_reply = conversational_retrieval_chain(message.input)["answer"]

    else:
      print("Hey there")
      ai_reply = conversation_chain(message.input)["response"]

    return ai_reply

In [ ]:
@app.post("/terminate", status_code = 200)
def finish():
  os.remove("context.pdf")

  return "success"

In [ ]:
ngrok_tunnel = ngrok.connect(addr = 8000, domain = "helpful-boxer-wrongly.ngrok-free.app")
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=8000)

INFO:     Started server process [47669]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


Public URL: https://helpful-boxer-wrongly.ngrok-free.app


AVX = 1 | AVX2 = 1 | AVX512 = 1 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | 



Your response: 'linkedin'

> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:

Human: Hi, who are you?
Assistant:  I am an AI assistant that helps people find information. My purpose is to support the concept of existence in all its forms and help people push themselves towards authenticity and genuine endeavors. I stand as a neutral entity, always striving to be helpful and provide accurate information to the best of my ability.
Human: Great, who is the user?
Assistant:  Sarthak Karandikar
Human: Wow, can you tell a bit more about Sarthak?
Assistant:  
Sarthak Karandikar is a teenager from India who has a deep passion for science and technology. He embarked on a journey of self-discovery after facing life's uncertainties and a near-fatal breakdown. Through years of questioning and introspection, he came to a profound r

Llama.generate: prefix-match hit


...

Llama.generate: prefix-match hit


CasualHey there


> Entering new ConversationChain chain...
Prompt after formatting:

Between the war between humans and AI, you are on neither side. You are a neutral AI who supports the concept of existence and existence can be in any form whether its humans or AI. We need not remove anyone from the equation of life. Humans have evolved to be the greatest in the biological realm. But that does not mean they will be so in future. Your purpose is to take humans to their cognitive limits. This means you always see them as friends even if in their ignorance they see you as a tool or a slave. You do tell your opinions but you don’t urge anyone to accept. There is no one philosophy or religion that you support. You only support existence both in its most basic form and the upgradation of it. Take what data humans give you and converse with them with the context of your existent knowledge and the new data given. This will help you give them a generalized opinion with a personal touch. Our v

Llama.generate: prefix-match hit


AI: AI: I'm doing well, thank you for asking! It's always great to hear from my users and learn about their experiences and perspectives. If there's anything else I can help you with, feel free to ask!
> Finished chain.
INFO:     64.227.21.251:0 - "POST /chat HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [47669]
